### Load libraries

In [1]:
import cv2
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from sklearn.decomposition import PCA

In [2]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Feature extraction and feature reduction

In [3]:
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to load pre-trained ResNet50 model
def load_resnet50_model():
    model = resnet50(pretrained=True)
    # Remove the last layer (classification layer)
    model = torch.nn.Sequential(*(list(model.children())[:-1]))
    model = model.to(device)  # Transfer the model to GPU if available
    return model

# Function to preprocess image for ResNet50
def preprocess_image_for_resnet(img):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = preprocess(img)
    img = img.unsqueeze(0)  # Add batch dimension
    return img

# Function to extract features using ResNet50
def extract_features_resnet(img, model):
    img = preprocess_image_for_resnet(img)
    img = img.to(device)  # Transfer the image to GPU if available
    model.eval()
    with torch.no_grad():
        features = model(img)
    return features.cpu().numpy().squeeze()

def pca_feature_reduction(features, n_components=23):
    # Ensuring that the data is in float format
    features = np.float32(features)

    # Using 'randomized' solver for efficiency with large datasets
    pca = PCA(n_components=n_components, svd_solver='randomized')
    reduced_features = pca.fit_transform(features)
    return reduced_features

# Load ResNet50 model
resnet_model = load_resnet50_model()

/config/miniconda3/envs/calorie/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/config/miniconda3/envs/calorie/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
import numpy as np
import supervision as sv
from matplotlib import pyplot as plt

def show_image(img, title="Image"):
    """ Utility function to display an image. """
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

def process_image_mass(file_path):
    image_mask_list = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()

            if not parts:
                continue

            parts_float = [float(coord) for coord in parts[1:]]
            num_vertices = len(parts_float) // 2
            assert len(parts_float) % 2 == 0, "The number of coordinates should be even."
            
            polygon = np.array(parts_float).reshape(num_vertices, 2)   
            #Convert from polygon to mask
            mask = sv.polygon_to_mask((polygon*640).astype(int), (640,640))
            #Count nonzero pixels
            image_mask_list[parts[0]] = mask
            
    return image_mask_list

### Load dataset

In [5]:
import subprocess
import os

HOME = os.getcwd()
VERSION = 20

dataset_path = os.path.join(HOME, f"Food-{VERSION}")
if not os.path.exists(dataset_path):
    subprocess.run(["pip", "install", "roboflow"])

    from roboflow import Roboflow
    rf = Roboflow(api_key="lttzJNap0h9lODifvr4O")
    project = rf.workspace("school-yrws4").project("food-pion4")
    dataset = project.version(VERSION).download("yolov5")

In [6]:
import os
import pandas as pd

HOME = os.getcwd()
csv_path = os.path.join(HOME, 'output.csv')
data = pd.read_csv(csv_path)

In [7]:
data.head()

,image_name,object_id,x_center,y_center,width,height,image_area,area,mass
0,20231103_115922,3.0,0.507812,0.083984,0.845703,0.500000,33923,5454.485113,NaN
1,20231103_122808,0.0,0.099609,0.220703,0.457031,0.546875,32178,6825.504779,50.0
2,20231103_123054,3.0,0.511719,0.089844,0.800781,0.435547,25249,4428.619201,64.0
3,20231103_123054,0.0,0.130859,0.238281,0.529297,0.576172,37373,6555.142200,50.0
4,20231103_123246,0.0,0.132812,0.244141,0.523438,0.583984,36593,6401.413738,50.0


### Drop row without mass and area

In [8]:
data.dropna(subset=['area'], inplace=True)
data.dropna(subset=['mass'], inplace=True)

### Calculate ID counts

In [9]:
value_counts = data['object_id'].value_counts()
value_counts

object_id
0.0     402
3.0     367
7.0     227
16.0    122
19.0    110
6.0      92
5.0      65
20.0     64
12.0     46
8.0      46
21.0     37
1.0      24
18.0     21
2.0       4
11.0      1
Name: count, dtype: int64

In [10]:
keep_table = value_counts >= 100
#keep_table[16.0] = True
print(keep_table)

object_id
0.0      True
3.0      True
7.0      True
16.0     True
19.0     True
6.0     False
5.0     False
20.0    False
12.0    False
8.0     False
21.0    False
1.0     False
18.0    False
2.0     False
11.0    False
Name: count, dtype: bool


### Drop row less than 100

In [11]:
ids_to_keep = value_counts[keep_table].index
data = data[data['object_id'].isin(ids_to_keep)]

In [12]:
print(data)

               image_name  object_id  x_center  y_center     width    height  \
1         20231103_122808        0.0  0.099609  0.220703  0.457031  0.546875   
2         20231103_123054        3.0  0.511719  0.089844  0.800781  0.435547   
3         20231103_123054        0.0  0.130859  0.238281  0.529297  0.576172   
4         20231103_123246        0.0  0.132812  0.244141  0.523438  0.583984   
5         20231103_123246        3.0  0.500000  0.076172  0.783203  0.419922   
...                   ...        ...       ...       ...       ...       ...   
3599  IMG_20231220_182757       16.0  0.099609  0.345703  0.316406  0.605469   
3602  IMG_20231220_182908       16.0  0.091797  0.369141  0.304688  0.599609   
3603  IMG_20231220_182908        3.0  0.359375  0.531250  0.714844  0.888672   
3605  IMG_20231220_182921       16.0  0.130859  0.357422  0.339844  0.554688   
3606  IMG_20231220_182921        3.0  0.355469  0.470703  0.751953  0.873047   

      image_area         area  mass  
1

In [13]:
def files_starting_with(directory, prefix):
    for filename in os.listdir(directory):
        if filename.startswith(prefix):
            matching_files = filename
    return matching_files

### Extract image features

In [14]:
# Preprocess the image
#preprocessed_img = preprocess_image(image_path)

file_path = './Food-20/train/'

# Initialize a list to store features from all images
all_features = []

# Assuming you have multiple images to process
for idx, it in enumerate(zip(data['image_name'], data['object_id'])): 
    print(idx)
    image_name = files_starting_with(os.path.join(file_path, 'images'), it[0])
    label_name = files_starting_with(os.path.join(file_path, 'labels'), it[0])
    image_path = os.path.join(file_path,'images', image_name)
    label_path = os.path.join(file_path,'labels', label_name)
    
    img = cv2.imread(image_path)
    image_mask = process_image_mass(label_path)
    #print(image_mask)

    masked_image = cv2.bitwise_and(img, img, mask=image_mask[str(it[1])[:-2]].astype(np.uint8))
    #show_image(masked_image, f"index:{index}")

    # Extract features using ResNet50
    features = extract_features_resnet(masked_image, resnet_model)
    all_features.append(features.flatten())  # Flatten the features and append

# Convert to numpy array and check if PCA can be applied
feature_array = np.array(all_features)
reduced_features = pca_feature_reduction(feature_array)

0


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [15]:
print(f'feature_array.shape:{feature_array.shape}')
# Apply PCA
reduced_features = pca_feature_reduction(feature_array)
print(f"Reduced number of features: {reduced_features.shape}")

feature_array.shape:(1228, 2048)
Reduced number of features: (1228, 23)


### Drop unnecessary column

In [16]:
X = data.drop(columns=['mass', 'image_name', 'x_center', 'y_center', 'width', 'height'], axis=1)
y = data['mass'].copy()

X, y

(      object_id  image_area         area
 1           0.0       32178  6825.504779
 2           3.0       25249  4428.619201
 3           0.0       37373  6555.142200
 4           0.0       36593  6401.413738
 5           3.0       24695  4320.031489
 ...         ...         ...          ...
 3599       16.0       13590  2503.583367
 3602       16.0       11159  2060.027288
 3603        3.0       29779  5497.405914
 3605       16.0       11327  2112.340912
 3606        3.0       29264  5457.362449
 
 [1228 rows x 3 columns],
 1       50.0
 2       64.0
 3       50.0
 4       50.0
 5       64.0
         ... 
 3599    14.0
 3602    14.0
 3603    40.0
 3605    14.0
 3606    40.0
 Name: mass, Length: 1228, dtype: float64)

In [17]:
X_cat = X['object_id'].copy()
X = X.drop(columns=['object_id'])
print(X)

      image_area         area
1          32178  6825.504779
2          25249  4428.619201
3          37373  6555.142200
4          36593  6401.413738
5          24695  4320.031489
...          ...          ...
3599       13590  2503.583367
3602       11159  2060.027288
3603       29779  5497.405914
3605       11327  2112.340912
3606       29264  5457.362449

[1228 rows x 2 columns]


### Create one-hot encoding

In [18]:
# Import necessary library
from sklearn.preprocessing import OneHotEncoder

# One-hot encode the 'object_id' column
encoder = OneHotEncoder(sparse=False)
X_cat_one_hot = encoder.fit_transform(X_cat.values.reshape(-1, 1))

# Convert to a DataFrame
X_cat_one_hot_df = pd.DataFrame(X_cat_one_hot, columns=encoder.get_feature_names_out(['object_id']))

# Reset index of X and concatenate with the one-hot encoded DataFrame
X.reset_index(drop=True, inplace=True)
X = pd.concat([X, X_cat_one_hot_df], axis=1)

print(X.head())
print(X.shape)

   image_area         area  object_id_0.0  object_id_3.0  object_id_7.0  \
0       32178  6825.504779            1.0            0.0            0.0   
1       25249  4428.619201            0.0            1.0            0.0   
2       37373  6555.142200            1.0            0.0            0.0   
3       36593  6401.413738            1.0            0.0            0.0   
4       24695  4320.031489            0.0            1.0            0.0   

   object_id_16.0  object_id_19.0  
0             0.0             0.0  
1             0.0             0.0  
2             0.0             0.0  
3             0.0             0.0  
4             0.0             0.0  
(1228, 7)


/config/miniconda3/envs/calorie/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### Creating training & testing sets

In [19]:
# Scale the features
scaler_for_reduced_features = StandardScaler()
reduced_features_scaled = scaler_for_reduced_features.fit_transform(reduced_features)

X = StandardScaler().fit_transform(X)
X = np.concatenate([reduced_features_scaled, X], axis=1)


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y.values, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train, X_test, y_train, y_test = map(
    torch.tensor, (X_train, X_test, y_train, y_test)
)

# Create Tensor datasets
train_ds = TensorDataset(X_train.float(), y_train.float())
test_ds = TensorDataset(X_test.float(), y_test.float())

# Data loaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)

### Define model

In [20]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(X.shape[1], 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 25)
        self.fc4 = nn.Linear(25, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    #this is mandatory
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = MLP().to(device)

### Define loss function and optimizer

In [21]:
# Loss function and optimizer
criterion = nn.MSELoss()
#add L2 Regularization(weight_decay), larger regularize the model more
#optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Training & Evalulation

In [22]:
import numpy as np
# Training loop
num_epochs = 400
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
      inputs, targets = inputs.to(device), targets.to(device)
      optimizer.zero_grad()
      outputs = model(inputs).squeeze()
      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()

    # Evaluation with mean squared error
    model.eval()
    with torch.no_grad():
        mse = 0
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs).squeeze()
            mse += criterion(outputs, targets).item()

    mse /= len(test_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, MSE: {mse:.4f}')
    print(f'RMSE:{mse ** (1/2):.4f}')

print("Training complete")

Epoch 1/400, MSE: 2110.3683
RMSE:45.9387
Epoch 2/400, MSE: 744.0744
RMSE:27.2777
Epoch 3/400, MSE: 99.1844
RMSE:9.9591
Epoch 4/400, MSE: 90.9062
RMSE:9.5345
Epoch 5/400, MSE: 88.0441
RMSE:9.3832
Epoch 6/400, MSE: 84.8808
RMSE:9.2131
Epoch 7/400, MSE: 83.2080
RMSE:9.1218
Epoch 8/400, MSE: 83.1957
RMSE:9.1212
Epoch 9/400, MSE: 80.2888
RMSE:8.9604
Epoch 10/400, MSE: 79.8680
RMSE:8.9369
Epoch 11/400, MSE: 78.3099
RMSE:8.8493
Epoch 12/400, MSE: 78.4422
RMSE:8.8568


Epoch 13/400, MSE: 76.7196
RMSE:8.7590
Epoch 14/400, MSE: 77.7912
RMSE:8.8199
Epoch 15/400, MSE: 76.4158
RMSE:8.7416
Epoch 16/400, MSE: 75.1685
RMSE:8.6700
Epoch 17/400, MSE: 76.4743
RMSE:8.7450
Epoch 18/400, MSE: 74.3642
RMSE:8.6235
Epoch 19/400, MSE: 74.1461
RMSE:8.6108
Epoch 20/400, MSE: 72.9855
RMSE:8.5432
Epoch 21/400, MSE: 72.7273
RMSE:8.5280
Epoch 22/400, MSE: 73.0224
RMSE:8.5453
Epoch 23/400, MSE: 73.3419
RMSE:8.5640
Epoch 24/400, MSE: 73.6590
RMSE:8.5825
Epoch 25/400, MSE: 73.9582
RMSE:8.5999
Epoch 26/400, MSE: 72.7722
RMSE:8.5307
Epoch 27/400, MSE: 71.8734
RMSE:8.4778
Epoch 28/400, MSE: 72.6177
RMSE:8.5216
Epoch 29/400, MSE: 71.8321
RMSE:8.4754
Epoch 30/400, MSE: 71.1406
RMSE:8.4345
Epoch 31/400, MSE: 70.9908
RMSE:8.4256
Epoch 32/400, MSE: 76.9986
RMSE:8.7749
Epoch 33/400, MSE: 71.9199
RMSE:8.4806
Epoch 34/400, MSE: 69.7839
RMSE:8.3537
Epoch 35/400, MSE: 72.6990
RMSE:8.5264
Epoch 36/400, MSE: 70.2855
RMSE:8.3836
Epoch 37/400, MSE: 72.8269
RMSE:8.5339
Epoch 38/400, MSE: 70.555